In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import shap
import joblib

📌 2️⃣ Load + Combine All Ranked CSVs

In [19]:
data_path = "../data/ranked_data"

files = sorted([f for f in os.listdir(data_path) if f.endswith(".csv")])

dfs = []
for f in files:
    year = int("".join(filter(str.isdigit, f)))  # extracts 2018, 2019, etc.
    df = pd.read_csv(os.path.join(data_path, f))
    df["Year"] = year
    dfs.append(df)

nirf_all = pd.concat(dfs, ignore_index=True)
nirf_all.head()

,Institute Name,Institute Code,SS,FSR,FQE,FRU,PU,QP,IPR,FPPP,...,RD,WD,ESCS,PCS,PR,Rank,Year,Institution Name,Institution Code,OE
0,Indian Institute of Science,IR-1-O-O-U-0220,10.25,30.00,18.30,26.00,35.00,35.00,14.0,7.08,...,21.12,9.50,4.08,9.0,100,1.0,2018,NaN,NaN,NaN
1,Indian Institute of Technology Madras,IR-2-O-OE-U-0456,17.00,27.77,17.92,19.00,28.28,28.93,13.0,11.21,...,22.86,13.37,8.57,20.0,86.61,2.0,2018,NaN,NaN,NaN
2,Indian Institute of Technology Bombay,IR-3-O-OEM-U-0306,17.00,28.55,19.20,16.59,29.83,31.68,15.0,9.08,...,22.23,13.23,6.07,7.0,79.61,3.0,2018,NaN,NaN,NaN
3,Indian Institute of Technology Delhi,IR-3-O-OEM-I-1074,14.85,24.41,15.89,18.22,29.61,31.18,9.0,8.88,...,23.01,13.65,4.30,20.0,72.79,4.0,2018,NaN,NaN,NaN
4,Indian Institute of Technology Kharagpur,IR-5-O-OEMAL-U-0573,16.56,23.25,15.21,12.95,28.36,32.47,7.0,6.74,...,22.39,10.71,3.06,20.0,62.12,5.0,2018,NaN,NaN,NaN


📌 3️⃣ Clean Data (Convert to Numeric, Fix NaNs)

In [20]:
clean_df = nirf_all.copy()

for col in clean_df.columns:
    clean_df[col] = pd.to_numeric(clean_df[col], errors="ignore")

num_cols = clean_df.select_dtypes(include=[np.number]).columns

clean_df[num_cols] = clean_df[num_cols].fillna(clean_df[num_cols].median())

clean_df.head()

/var/folders/03/d12ppvv967x43mxw8d91d2q80000gn/T/ipykernel_11961/4142447704.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  clean_df[col] = pd.to_numeric(clean_df[col], errors="ignore")


,Institute Name,Institute Code,SS,FSR,FQE,FRU,PU,QP,IPR,FPPP,...,RD,WD,ESCS,PCS,PR,Rank,Year,Institution Name,Institution Code,OE
0,Indian Institute of Science,IR-1-O-O-U-0220,10.25,30.00,18.30,26.00,35.00,35.00,14.0,7.08,...,21.12,9.50,4.08,9.0,100,1.0,2018,NaN,NaN,0.0
1,Indian Institute of Technology Madras,IR-2-O-OE-U-0456,17.00,27.77,17.92,19.00,28.28,28.93,13.0,11.21,...,22.86,13.37,8.57,20.0,86.61,2.0,2018,NaN,NaN,0.0
2,Indian Institute of Technology Bombay,IR-3-O-OEM-U-0306,17.00,28.55,19.20,16.59,29.83,31.68,15.0,9.08,...,22.23,13.23,6.07,7.0,79.61,3.0,2018,NaN,NaN,0.0
3,Indian Institute of Technology Delhi,IR-3-O-OEM-I-1074,14.85,24.41,15.89,18.22,29.61,31.18,9.0,8.88,...,23.01,13.65,4.30,20.0,72.79,4.0,2018,NaN,NaN,0.0
4,Indian Institute of Technology Kharagpur,IR-5-O-OEMAL-U-0573,16.56,23.25,15.21,12.95,28.36,32.47,7.0,6.74,...,22.39,10.71,3.06,20.0,62.12,5.0,2018,NaN,NaN,0.0


📌 4️⃣ Save Combined File

In [21]:
save_path = "../data/processed/nirf_combined.csv"
clean_df.to_csv(save_path, index=False)
save_path

'../data/processed/nirf_combined.csv'

📌 5️⃣ Train Model (RF + XGB → auto choose best)

In [22]:
target = "Rank"
features = [col for col in clean_df.columns 
            if col not in ["Rank", "Institution Name", "Institution Code", "Year"]]

X = clean_df[features]
y = clean_df[target]

In [24]:
drop_cols = ["Institute Name", "Institute Code", "Rank", "Year"]

In [25]:
numeric_cols = clean_df.drop(columns=drop_cols, errors="ignore").select_dtypes(include=['int64','float64']).columns.tolist()

X = clean_df[numeric_cols]
y = clean_df["Rank"]

In [26]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [27]:
print("Shape:", X.shape)
print("Dtypes:\n", X.dtypes)

Shape: (501, 15)
Dtypes:
 SS      float64
FSR     float64
FQE     float64
FRU     float64
PU      float64
QP      float64
IPR     float64
FPPP    float64
GUE     float64
GPHD    float64
RD      float64
WD      float64
ESCS    float64
PCS     float64
OE      float64
dtype: object


In [28]:
joblib.dump(best_model, "../models/nirf_pr_model.pkl")
joblib.dump(scaler, "../models/nirf_scaler.pkl")

NameError: name 'best_model' is not defined

In [29]:
try:
    best_model = grid.best_estimator_
except:
    try:
        best_model = random_search.best_estimator_
    except:
        best_model = model

NameError: name 'model' is not defined

In [30]:
best_model = grid.best_estimator_

NameError: name 'grid' is not defined

In [34]:
# Make sure you have one of these defined before saving
try:
    best_model = grid.best_estimator_
except NameError:
    try:
        best_model = random_search.best_estimator_
    except NameError:
        # If you have a trained model variable, use that instead
        best_model = your_trained_model  # Replace with your actual model variable

# Then save the model
joblib.dump(best_model, "../models/nirf_pr_model.pk")
joblib.dump(scaler, "../models/nirf_scaler.pk")

NameError: name 'your_trained_model' is not defined

In [35]:
# Check what variables are available
print("Available variables:", [var for var in dir() if not var.startswith('_')])

# Look for your model variables - common names might be:
# model, clf, classifier, regressor, grid, random_search, etc.

# Then use the appropriate one
if 'grid' in dir():
    best_model = grid.best_estimator_
elif 'random_search' in dir():
    best_model = random_search.best_estimator_
elif 'model' in dir():
    best_model = model
else:
    print("No model found! Please train a model first.")

Available variables: ['In', 'Out', 'RandomForestRegressor', 'StandardScaler', 'X', 'XGBRegressor', 'X_scaled', 'clean_df', 'col', 'data_path', 'df', 'dfs', 'drop_cols', 'exit', 'f', 'features', 'files', 'get_ipython', 'joblib', 'load_nirf_csv', 'load_nirf_file', 'nirf_all', 'np', 'num_cols', 'numeric_cols', 'open', 'os', 'pd', 'plt', 'quit', 'save_path', 'scaler', 'shap', 'target', 'train_test_split', 'y', 'year']
No model found! Please train a model first.


In [ ]:
# Your model training code should come before the saving code
# For example:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define and train your model
model = RandomForestRegressor()
# ... training code ...
grid = GridSearchCV(model, param_grid, cv=5)
grid.fit(X_train, y_train)

# Now you can save
best_model = grid.best_estimator_
joblib.dump(best_model, "../models/nirf_pr_model.pk")
joblib.dump(scaler, "../models/nirf_scaler.pk")